In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import math
# import wurlitzer

# try:
#   from wurlitzer import sys_pipes
# except:
#   from colabtools.googlelog import CaptureLog as sys_pipes

from IPython.core.magic import register_line_magic
from IPython.display import Javascript

Load Dataset

In [2]:
file_path = '/content/Busaha_Dataset.csv'
try:
  df = pd.read_csv(file_path)
  print("File Loaded!")
except:
  !gdown --id 1dz0_0AsxbyNTJUQGWX0ZjTLcPZMfEYS7
  try:
    df = pd.read_csv(file_path)
    print("File Loaded!")
  except:
    print("Something's wrong! check your link!!!")

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1dz0_0AsxbyNTJUQGWX0ZjTLcPZMfEYS7
To: /content/Busaha_Dataset.csv
100% 45.0k/45.0k [00:00<00:00, 44.5MB/s]
File Loaded!


Checking Dataset

In [3]:
df.head()

,Unnamed: 0,jenis_usaha,Modal,modal_minimum,modal_maximum,tingkat_persaingan,tingkat_persaingan_idx,jumlah_pegawai,pegawai_minimum,omset,...,sosial_media,sosial_media_idx,barang_toserba,toko_toserba,toko_toserba_idx,tipe_toko,tipe_toko_idx,usaha_lain,tipe_usaha_lain,tipe_usaha_lain_idx
0,0,Transportasi,> 25jt,25000000.0,100000000.0,Sedang,1,2-4 orang,2,> 50 Juta,...,no_info,0,no_info,no_info,0,no_info,0,jasa sewa kendaraan,EX,1
1,1,Perternakan,10 - 25jt,10000000.0,25000000.0,Sedang,1,1 orang (Anda sendiri),1,10-50 Juta,...,no_info,0,no_info,no_info,0,no_info,0,no_info,no_info,0
2,2,Toserba/Toko Kelontong,1jt - 10 jt,1000000.0,10000000.0,Sedang,1,1 orang (Anda sendiri),1,< 10 Juta,...,no_info,0,Sembako,True,1,Ruko,1,no_info,no_info,0
3,3,Kuliner,1jt - 10 jt,1000000.0,10000000.0,Sedang,1,1 orang (Anda sendiri),1,< 10 Juta,...,no_info,0,no_info,no_info,0,no_info,0,no_info,no_info,0
4,4,Dropshipper,1jt - 10 jt,1000000.0,10000000.0,Rendah,0,1 orang (Anda sendiri),1,< 10 Juta,...,WA,1,no_info,no_info,0,no_info,0,no_info,no_info,0


The dataset doesn't look very good to be a model, so it needs to be preprocessed before being used as a model

In [4]:
# Deleting unused row
delrow = ["Transportasi", "Toko Buah", "Penggilingan", "Alat Kecantikan", "Koleksi",
          "Alat kesehatan", "Perseroan Komanditer", "Jasa Isi Ulang Air Minum", "Jasa Mengajar", "Fotocopy"
          ]
for row in delrow:
  df.drop(df[df['jenis_usaha'] == row].index, inplace = True)

In [5]:
# Deleting unused column
try:
  df = df.drop(columns=['Unnamed: 0','Modal','tingkat_persaingan','jumlah_pegawai','omset',
                      'target_pasar','lokasi','hewan_ternak','hewan_ternak_sub','lokasi_geografis',
                      'mengerti_hewan','luas_lahan','usaha_kuliner','tipe_usaha_kuliner','kemampuan_masak',
                      'barang_dropship','penyimpanan_barang','biaya_internet','sosial_media','barang_toserba',
                      'toko_toserba','tipe_toko','usaha_lain','tipe_usaha_lain'])
  print("Columns Dropped!")
except:
  print("Columns Already Dropped!")

Columns Dropped!


In [6]:
df.head()

,jenis_usaha,modal_minimum,modal_maximum,tingkat_persaingan_idx,pegawai_minimum,omset_minimum,tp_b,tp_m,tp_a,lokasi_idx,...,mengerti_hewan_idx,lahan_minimum,tipe_usaha_kuliner_idx,kemampuan_masak_idx,penyimpanan_barang_idx,internet_minimum,sosial_media_idx,toko_toserba_idx,tipe_toko_idx,tipe_usaha_lain_idx
1,Perternakan,10000000.0,25000000.0,1,1,25000000.0,1,1,1,1,...,1,1000.0,0,0,0,0,0,0,0,0
2,Toserba/Toko Kelontong,1000000.0,10000000.0,1,1,1000000.0,1,1,0,1,...,0,0.0,0,0,0,0,0,1,1,0
3,Kuliner,1000000.0,10000000.0,1,1,1000000.0,0,1,0,1,...,0,0.0,1,1,0,0,0,0,0,0
4,Dropshipper,1000000.0,10000000.0,0,1,1000000.0,1,0,0,0,...,0,0.0,0,0,1,100000,1,0,0,0
5,Dropshipper,1000000.0,10000000.0,2,1,1000000.0,1,1,0,0,...,0,0.0,0,0,1,300000,2,0,0,0


Indexing Label

In [7]:
label = "jenis_usaha"
classes = [0]

In [8]:
if type(classes[0]) is int:
  classes = df[label].unique().tolist()
  df[label] = df[label].map(classes.index)
  print(f"Label classes: {classes}")
else:
  print("Label classes Already indexed!")
  print(f"Label classes: {classes}")

Label classes: ['Perternakan', 'Toserba/Toko Kelontong', 'Kuliner', 'Dropshipper', 'Barang Digital', 'Elektronik', 'Pakaian', 'Kosmetik']


In [9]:
# # Inappropriate data normalization
# from sklearn.preprocessing import normalize
# df_to_normalize = df[['modal_minimum', 'modal_maximum', 'omset_minimum']]
# df[['modal_minimum', 'modal_maximum', 'omset_minimum']] = normalize(df_to_normalize, axis=0)
print(len(classes))

8


In [10]:
df.head()

,jenis_usaha,modal_minimum,modal_maximum,tingkat_persaingan_idx,pegawai_minimum,omset_minimum,tp_b,tp_m,tp_a,lokasi_idx,...,mengerti_hewan_idx,lahan_minimum,tipe_usaha_kuliner_idx,kemampuan_masak_idx,penyimpanan_barang_idx,internet_minimum,sosial_media_idx,toko_toserba_idx,tipe_toko_idx,tipe_usaha_lain_idx
1,0,10000000.0,25000000.0,1,1,25000000.0,1,1,1,1,...,1,1000.0,0,0,0,0,0,0,0,0
2,1,1000000.0,10000000.0,1,1,1000000.0,1,1,0,1,...,0,0.0,0,0,0,0,0,1,1,0
3,2,1000000.0,10000000.0,1,1,1000000.0,0,1,0,1,...,0,0.0,1,1,0,0,0,0,0,0
4,3,1000000.0,10000000.0,0,1,1000000.0,1,0,0,0,...,0,0.0,0,0,1,100000,1,0,0,0
5,3,1000000.0,10000000.0,2,1,1000000.0,1,1,0,0,...,0,0.0,0,0,1,300000,2,0,0,0


In [11]:
# # Randomize data
# random = df.sample(frac=1, random_state=25)
# # separating labels and features
# X = random.drop("jenis_usaha", axis=1).values
# y = random.jenis_usaha.values

In [12]:
# separating labels and features
X = df.drop("jenis_usaha", axis=1).values
y = df.jenis_usaha.values

In [13]:
# this is for all normalization
from sklearn.preprocessing import normalize
X_normalized = normalize(X, axis = 0)

In [14]:
# this is for all normalization

total_length=len(df)
train_length=int(0.7*total_length)
test_length=int(0.3*total_length)

X_train=X_normalized[:train_length]
X_test=X_normalized[train_length:]
y_train=y[:train_length]
y_test=y[train_length:]

print("Length of train set x:",X_train.shape[0],"y:",y_train.shape[0])
print("Length of test set x:",X_test.shape[0],"y:",y_test.shape[0])

Length of train set x: 86 y: 86
Length of test set x: 37 y: 37


In [15]:
# # Separating training data and testing data
# total_length=len(df)
# train_length=int(0.8*total_length)
# test_length=int(0.2*total_length)

# X_train=X[:train_length]
# X_test=X[train_length:]
# y_train=y[:train_length]
# y_test=y[train_length:]

# print("Length of train set x:",X_train.shape[0],"y:",y_train.shape[0])
# print("Length of test set x:",X_test.shape[0],"y:",y_test.shape[0])

In [16]:
from keras.utils import np_utils
y_train=np_utils.to_categorical(y_train,num_classes=len(classes))
y_test=np_utils.to_categorical(y_test,num_classes=len(classes))
print("Shape of y_train",y_train.shape)
print("Shape of y_test",y_test.shape)

Shape of y_train (86, 8)
Shape of y_test (37, 8)


In [17]:
# from keras.models import Sequential 
# from keras.layers import Dense,Activation,Dropout 

# shape = X_train.shape[1]

# model=Sequential()
# model.add(Dense(shape*18,input_dim=X_train.shape[1],activation='relu'))
# model.add(Dense(shape*15,activation='relu'))
# model.add(Dense(shape*12,activation='relu'))
# model.add(Dense(shape*9,activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(18,activation='softmax'))
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [18]:
from keras.models import Sequential 
from keras.layers import Dense,Activation,Dropout 

shape = X_train.shape[1]

model=Sequential()
model.add(Dense(128,input_dim=X_train.shape[1],activation='relu'))
# model.add(Dense(256,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(classes),activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [19]:
class cb(tf.keras.callbacks.Callback): 
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > 0.9 and logs.get('val_loss') < 0.55 and logs.get('loss') < 0.55):
      # print("\nAkurasi telah mencapai di atas 97%, training berhenti")
      self.model.stop_training = True #training akan berhenti ketika accuracy dan val_accuracy telah mencapai >= 97%

callbacks = cb()

In [20]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),batch_size=20,epochs=100,verbose=1, callbacks=[callbacks])

Epoch 1/100
5/5 [==============================] - 1s 97ms/step - loss: 2.0743 - accuracy: 0.1395 - val_loss: 2.0485 - val_accuracy: 0.5405
Epoch 2/100
5/5 [==============================] - 0s 20ms/step - loss: 2.0395 - accuracy: 0.6047 - val_loss: 2.0093 - val_accuracy: 0.9459
Epoch 3/100
5/5 [==============================] - 0s 19ms/step - loss: 2.0095 - accuracy: 0.6279 - val_loss: 1.9748 - val_accuracy: 0.9459
Epoch 4/100
5/5 [==============================] - 0s 18ms/step - loss: 1.9749 - accuracy: 0.6279 - val_loss: 1.9375 - val_accuracy: 0.7568
Epoch 5/100
5/5 [==============================] - 0s 22ms/step - loss: 1.9319 - accuracy: 0.6744 - val_loss: 1.8917 - val_accuracy: 0.9459
Epoch 6/100
5/5 [==============================] - 0s 24ms/step - loss: 1.8899 - accuracy: 0.6860 - val_loss: 1.8373 - val_accuracy: 0.8378
Epoch 7/100
5/5 [==============================] - 0s 25ms/step - loss: 1.8437 - accuracy: 0.6860 - val_loss: 1.7719 - val_accuracy: 0.7838
Epoch 8/100
5/5 [===

In [21]:
fitur = [100000, 25000000, 1, 2, 10000000,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0]
kuliner = [1000000,10000000,1,1,10000000,0,1,0,1,0,0,0,0,1,1,0,0,0,0,0,0]
peternak = [100000, 25000000,1,1,10000000,1,1,1,1,1,1,1,1000,0,0,0,0,0,0,0,0]
elektronik = [100000,25000000,1,2,10000000,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,2]
Transportasi = [25000000, 100000000, 1, 2, 50000000,0,1,1,1,0,0,0,0,0,0, 0,0,0,0,0,1]

In [22]:
fitur_normalized = normalize([fitur], axis = 0)
prediction = model.predict([fitur_normalized])

clas = np.argmax(prediction, axis = 1)
clas1 = np.argsort(clas)
print(clas)
indextion = np.argsort((-prediction), axis = 1)[0,:3]
top1, top2, top3 = indextion
indextion0 = np.argsort((-prediction), axis = 1)[0]
indextion1 = np.argmax(prediction, axis = 1)
indextion2 = np.argmin(prediction)
print(f"top 3 prediksi adalah : {indextion}")
print(f"keseluruhan prediksi adalah : {indextion0}")
print(f"prediksi paling mendekati adalah : {indextion1}")
print(f"prediksi paling tidak mendekati adalah : {indextion2}")
print(f"label top 3 prediksi adalah : {classes[top1]}, {classes[top2]}, {classes[top3]}")

[2]
top 3 prediksi adalah : [2 1 6]
keseluruhan prediksi adalah : [2 1 6 5 7 4 3 0]
prediksi paling mendekati adalah : [2]
prediksi paling tidak mendekati adalah : 0
label top 3 prediksi adalah : Kuliner, Toserba/Toko Kelontong, Pakaian


In [23]:
fitur_normalized = normalize([kuliner], axis = 0)
prediction = model.predict([fitur_normalized])

clas = np.argmax(prediction, axis = 1)
clas1 = np.argsort(clas)
print(clas)
indextion = np.argsort((-prediction), axis = 1)[0,:3]
top1, top2, top3 = indextion
indextion0 = np.argsort((-prediction), axis = 1)[0]
indextion1 = np.argmax(prediction, axis = 1)
indextion2 = np.argmin(prediction)
print(f"top 3 prediksi adalah : {indextion}")
print(f"keseluruhan prediksi adalah : {indextion0}")
print(f"prediksi paling mendekati adalah : {indextion1}")
print(f"prediksi paling tidak mendekati adalah : {indextion2}")
print(f"label top 3 prediksi adalah : {classes[top1]}, {classes[top2]}, {classes[top3]}")

[2]
top 3 prediksi adalah : [2 1 5]
keseluruhan prediksi adalah : [2 1 5 6 7 3 4 0]
prediksi paling mendekati adalah : [2]
prediksi paling tidak mendekati adalah : 0
label top 3 prediksi adalah : Kuliner, Toserba/Toko Kelontong, Elektronik


In [24]:
fitur_normalized = normalize([peternak], axis = 0)
prediction = model.predict([fitur_normalized])

clas = np.argmax(prediction, axis = 1)
clas1 = np.argsort(clas)
print(clas)
indextion = np.argsort((-prediction), axis = 1)[0,:3]
top1, top2, top3 = indextion
indextion0 = np.argsort((-prediction), axis = 1)[0]
indextion1 = np.argmax(prediction, axis = 1)
indextion2 = np.argmin(prediction)
print(f"top 3 prediksi adalah : {indextion}")
print(f"keseluruhan prediksi adalah : {indextion0}")
print(f"prediksi paling mendekati adalah : {indextion1}")
print(f"prediksi paling tidak mendekati adalah : {indextion2}")
print(f"label top 3 prediksi adalah : {classes[top1]}, {classes[top2]}, {classes[top3]}")

[0]
top 3 prediksi adalah : [0 1 5]
keseluruhan prediksi adalah : [0 1 5 4 7 3 6 2]
prediksi paling mendekati adalah : [0]
prediksi paling tidak mendekati adalah : 2
label top 3 prediksi adalah : Perternakan, Toserba/Toko Kelontong, Elektronik


In [25]:
fitur_normalized = normalize([elektronik], axis = 0)
prediction = model.predict([fitur_normalized])

clas = np.argmax(prediction, axis = 1)
clas1 = np.argsort(clas)
print(clas)
indextion = np.argsort((-prediction), axis = 1)[0,:3]
top1, top2, top3 = indextion
indextion0 = np.argsort((-prediction), axis = 1)[0]
indextion1 = np.argmax(prediction, axis = 1)
indextion2 = np.argmin(prediction)
print(f"top 3 prediksi adalah : {indextion}")
print(f"keseluruhan prediksi adalah : {indextion0}")
print(f"prediksi paling mendekati adalah : {indextion1}")
print(f"prediksi paling tidak mendekati adalah : {indextion2}")
print(f"label top 3 prediksi adalah : {classes[top1]}, {classes[top2]}, {classes[top3]}")

[5]
top 3 prediksi adalah : [5 6 7]
keseluruhan prediksi adalah : [5 6 7 4 1 0 3 2]
prediksi paling mendekati adalah : [5]
prediksi paling tidak mendekati adalah : 2
label top 3 prediksi adalah : Elektronik, Pakaian, Kosmetik


In [26]:
fitur_normalized = normalize([Transportasi], axis = 0)
prediction = model.predict([fitur_normalized])

clas = np.argmax(prediction, axis = 1)
clas1 = np.argsort(clas)
print(clas)
indextion = np.argsort((-prediction), axis = 1)[0,:3]
top1, top2, top3 = indextion
indextion0 = np.argsort((-prediction), axis = 1)[0]
indextion1 = np.argmax(prediction, axis = 1)
indextion2 = np.argmin(prediction)
print(f"top 3 prediksi adalah : {indextion}")
print(f"keseluruhan prediksi adalah : {indextion0}")
print(f"prediksi paling mendekati adalah : {indextion1}")
print(f"prediksi paling tidak mendekati adalah : {indextion2}")
print(f"label top 3 prediksi adalah : {classes[top1]}, {classes[top2]}, {classes[top3]}")

[5]
top 3 prediksi adalah : [5 7 6]
keseluruhan prediksi adalah : [5 7 6 1 0 4 3 2]
prediksi paling mendekati adalah : [5]
prediksi paling tidak mendekati adalah : 2
label top 3 prediksi adalah : Elektronik, Kosmetik, Pakaian


In [27]:
model.save("/content/model")

INFO:tensorflow:Assets written to: /content/model/assets


In [28]:
new_model = tf.keras.models.load_model('/content/model')

In [29]:
fitur_normalized = normalize([elektronik], axis = 0)
pred = new_model.predict([fitur_normalized])
np.argmax(pred, axis=1)

array([5])

In [30]:
fitur_normalized = normalize([peternak], axis = 0)
pred = new_model.predict([fitur_normalized])
np.argmax(pred, axis=1)

array([0])

In [31]:
new_model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 13ms/step - loss: 0.0114 - accuracy: 1.0000


[0.011364714242517948, 1.0]

In [32]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model_busaha.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp30ggm1pq/assets
